In [0]:
# # Mount ADLS Gen2
# # Required each time the cluster is restarted which should be only on the first notebook as they run in order

# tiers = ["bronze", "silver", "gold"]
# adls_paths = {tier: f"abfss://{tier}@yagamiazdbsa.dfs.core.windows.net/" for tier in tiers}

# # Accessing paths
# bronze_adls = adls_paths["bronze"]
# silver_adls = adls_paths["silver"]
# gold_adls = adls_paths["gold"] 

# dbutils.fs.ls(bronze_adls)
# dbutils.fs.ls(silver_adls)
# dbutils.fs.ls(gold_adls)

In [0]:
# from datetime import date, timedelta
# start_date = date.today() - timedelta(days=1)
# end_date = date.today()

In [0]:
bronze_output = dbutils.jobs.taskValues.get(taskKey="bronze", key = "bronze_output")

start_date = bronze_output.get("start_date", "")
end_date = bronze_output.get("end_date", "")

bronze_adls = bronze_output.get("bronze_path", "")
silver_adls = bronze_output.get("silver_path", "")

print(f"Start date: {start_date}")
print(f"End date: {end_date}")
print(f"Bronze ADLS: {bronze_adls}")
print(f"Silver ADLS: {silver_adls}")

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-7012751062707529>, line 1
----> 1 bronze_output = dbutils.jobs.taskValues.get(taskKey="bronze", key = "bronze_output")
      3 start_date = bronze_output.get("start_date", "")
      4 end_date = bronze_output.get("end_date", "")

File /databricks/python_shell/lib/dbruntime/dbutils.py:254, in DBUtils.JobsHandler.TaskValuesHandler.get(self, taskKey, key, default, debugValue)
    252 elif 'NotInJobContextException' in exceptionClassName:
    253     if debugValue is None:
--> 254         raise TypeError(
    255             'Must pass debugValue when calling get outside of a job context. debugValue cannot be None.'
    256         ) from None
    257     return debugValue
    259 raise

TypeError: Must pass debugValue when calling get outside of a job context. debugValue cannot be None.

In [0]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")


In [0]:
# df.head()

In [0]:
# display(df)

In [0]:

# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [0]:
# display(df)

In [0]:

# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [0]:

# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [0]:
# display(df)

In [0]:

# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"

In [0]:

# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)

In [0]:
# Set task values for workflow
dbutils.jobs.taskValues.set(key="silver_output", value=silver_output_path)
print("Silver notebook completed successfully!")

